# ***Final project for Python***

# *Projet : Agence de location de voitture*

**Création** classe Vehicule


In [ ]:
class Vehicule :
    def __init__(self,immatriculation, marque, modele, kilometrage ,statut="Disponible"):
        self.immatriculation = immatriculation.strip().upper()
        self.marque = marque
        self.modele = modele
        self.kilometrage = kilometrage
        while True:

          if not statut:
              self.statut = "Disponible"

          elif statut.capitalize() in ["Disponible","Loue", "Loué", "Maintenance"]:
              self.statut = statut.capitalize()

          else:
              print(f"Statut invalide Inserez une valide statut [Disponible, Loué, Maintenance] :")
              statut = input("veuillez saisir le statut du vehicule :  ")
              continue

          break
    def to_dict(self):
        return {
            "immatriculation": self.immatriculation,
            "marque": self.marque,
            "modele": str(self.modele),
            "kilometrage": self.kilometrage,
            "statut": self.statut,
            "type": self.__class__.__name__  # pour reconstruire le type exact
        }

    def from_dict(data):
        if data["type"] == "VoitureTourisme":
            return VoitureTourisme(
                data["immatriculation"], data["marque"], data["modele"],
                data["kilometrage"], data["statut"], data.get("nombre_sieges", 5)
            )
        elif data["type"] == "Utilitaire":
            return Utilitaire(
                data["immatriculation"], data["marque"], data["modele"],
                data["kilometrage"], data["statut"], data.get("chargement", 0)
            )
        else:
            return Vehicule(
                data["immatriculation"], data["marque"], data["modele"],
                data["kilometrage"], data["statut"]
            )
    def __str__(self):
        return (f"[{self.__class__.__name__}] "
                f"Immatriculation: {self.immatriculation}, "
                f"Marque: {self.marque}, "
                f"Modèle: {self.modele}, "
                f"Kilométrage: {self.kilometrage}, "
                f"Statut: {self.statut}")

**Creation** des classes enfants VoitureTourisme et Utilitaire

In [ ]:
class VoitureTourisme(Vehicule):
    def __init__(self,immatriculation, marque, modele, kilometrage ,statut, nombre_sieges):
        self.nombre_sieges = nombre_sieges
        super().__init__(immatriculation, marque, modele, kilometrage ,statut)

    def to_dict(self):
        data = super().to_dict()
        data["nombre_sieges"] = self.nombre_sieges
        return data

    def from_dict(data):
        return VoitureTourisme(
            data["immatriculation"], data["marque"], data["modele"],
            data["kilometrage"], data["statut"], data.get("nombre_sieges", 5)
        )
    def __str__(self):
        return super().__str__() + f", Nombre de places: {self.nombre_sieges}"

class Utilitaire (Vehicule):
    def __init__(self,immatriculation, marque, modele, kilometrage ,statut,chargement):
        self.chargement = chargement
        super().__init__(immatriculation, marque, modele, kilometrage ,statut)
    def to_dict(self):
        data = super().to_dict()
        data["chargement"] = self.chargement
        return data

    def from_dict(data):
        return Utilitaire(
            data["immatriculation"], data["marque"], data["modele"],
            data["kilometrage"], data["statut"], data.get("chargement", 0)
        )
    def __str__(self):
        return super().__str__() + f", Capacité de chargement: {self.chargement} m3"

# **Creation** de classe Client

In [ ]:
class Client:
  compteur_id = 0
  def __init__(self, nom, numero_permis):
        self.nom = nom.strip().capitalize()
        self.numero_permis = numero_permis.strip().upper()
        Client.compteur_id = Client.compteur_id + 1
        self.id_client = Client.compteur_id
  def to_dict(self):
        return {
            "id_client": self.id_client,
            "nom": self.nom,
            "numero_permis": self.numero_permis
        }

  def from_dict(data):
        c = Client(data["nom"], data["numero_permis"])
        c.id_client = data["id_client"]
        return c
  def __str__(self):
        return f"Client(id={self.id_client}, nom='{self.nom}', numero_permis='{self.numero_permis}')"

# **Creation** du Class LocationActive

In [ ]:
class LocationActive:
    def __init__(self, Vehicule,Client,date_debut):
        self.Vehicule = Vehicule
        self.Client = Client
        self.date_debut = date_debut

    def to_dict(self):
        return {
            "immatriculation": self.Vehicule.immatriculation,
            "numero_permis": self.Client.numero_permis,
            "date_debut": self.date_debut.isoformat() if isinstance(self.date_debut, datetime.date) else self.date_debut
        }

    def from_dict(data, flot_voitures, clients):
        vehicule = flot_voitures[data["immatriculation"]]
        client = clients[data["numero_permis"]]
        date_debut = datetime.datetime.strptime(data["date_debut"], "%Y-%m-%d").date()
        return LocationActive(vehicule, client, date_debut)

    def __str__(self):
        return f"LocationActive(Vehicule={self.Vehicule}, Client={self.Client}, date_debut={self.date_debut})"

## **Creation** de classe Contrat de location

In [ ]:
class Contrat_Location:
    def __init__(self, id_contrat, immatriculation_du_vehicule, id_du_client,
                 date_debut, date_retour, kilometrage_parcouru, cout_total):
        self.id_contrat = id_contrat
        self.immatriculation_du_vehicule = immatriculation_du_vehicule
        self.id_du_client = id_du_client
        self.date_debut = date_debut
        self.date_retour = date_retour
        self.kilometrage_parcouru = kilometrage_parcouru
        self.cout_total = cout_total

    def to_dict(self):
        return {
            "id_contrat": self.id_contrat,
            "immatriculation_du_vehicule": self.immatriculation_du_vehicule,
            "id_du_client": self.id_du_client,
            "date_debut": self.date_debut.isoformat() if isinstance(self.date_debut, datetime.date) else self.date_debut,
            "date_retour": self.date_retour.isoformat() if isinstance(self.date_retour, datetime.date) else self.date_retour,
            "kilometrage_parcouru": self.kilometrage_parcouru,
            "cout_total": self.cout_total
        }

    @staticmethod
    def from_dict(data):
        date_debut = datetime.datetime.fromisoformat(data["date_debut"]).date() if isinstance(data["date_debut"], str) else data["date_debut"]
        date_retour = datetime.datetime.fromisoformat(data["date_retour"]).date() if isinstance(data["date_retour"], str) else data["date_retour"]
        return Contrat_Location(
            id_contrat=data["id_contrat"],
            immatriculation_du_vehicule=data["immatriculation_du_vehicule"],
            id_du_client=data["id_du_client"],
            date_debut=date_debut,
            date_retour=date_retour,
            kilometrage_parcouru=data["kilometrage_parcouru"],
            cout_total=data["cout_total"]
        )
    def __str__(self):
        return f"Contrat_Location(id_contrat={self.id_contrat}, immatriculation_du_vehicule={self.immatriculation_du_vehicule}, id_du_client={self.id_du_client}"

# Classe Main Agence

In [ ]:
from datetime import date
import json
import datetime

In [ ]:
class Agence:
    def __init__(self):
        self.flot_voitures = {}
        self.clients = {}
        self.location_actives = {}
        self.historique_contrats = {}

    def ajouterNouveauVehicule(self):
        print("veuillez saisir les informations du nouveau vehicule : ")
        while True:

            immatriculation = input("veuillez saisir l'immatriculation du vehicule :  ").strip().upper()

            if not immatriculation:
                print("Anserez valid immatriculation")
                continue
            if immatriculation in self.flot_voitures:
                print("vehicule deja existant")
                continue
            break

        while True:

            marque = input("veuillez saisir la marque du vehicule :  ").strip()

            if not marque:
                print("Anserez valid marque")
                continue
            break


        while True:
          modele = input("veuillez saisir le modele du vehicule :  ").strip()


          if len(modele) == 10 and modele[2] == "/" and modele[5] == "/":
              jj, mm, aaaa = modele.split("/")

              if jj.isdigit() and mm.isdigit() and aaaa.isdigit():
                jj, mm, aaaa = int(jj), int(mm), int(aaaa)

                try:
                    modele_date = date(aaaa, mm, jj)
                    if 1 <= jj <= 31 and 1 <= mm <= 12 and aaaa < date.today().year and modele_date < date.today():
                        modele = modele_date
                        break
                    else:
                        print("Entrez un modele valide")
                except ValueError:
                    print("Date invalide")
                    continue
          else:
              print("Format invalide, veuillez entrer un modele au format jj/mm/aaaa")




        while True:

            kilometrage = input("veuillez saisir le kilometrage du vehicule :  ").strip()

            if not kilometrage.isdigit():
                print("Anserez valid kilometrage")
                continue
            break

        statut = input("veuillez saisir le statut du vehicule :  ")



        while True:
            choix = input("Ce véhicule est-il (1) Tourisme ou (2) Utilitaire ? : ")
            if choix == "1":
                nb_sieges = input("Nombre de sièges : ")
                voiture = VoitureTourisme(immatriculation, marque, modele, kilometrage, statut, nb_sieges)
                break
            elif choix == "2":
                charge = input("Capacité de chargement (en m3) : ")
                voiture = Utilitaire(immatriculation, marque, modele, kilometrage, statut, charge)
                break
            else:
                print(" Choix invalide ! Veuillez taper 1 (Tourisme) ou 2 (Utilitaire).")

        self.flot_voitures[immatriculation] = voiture
        print(" Véhicule ajouté avec succès")
        return voiture

    def afficherFlotVehicules(self):
        print("-------------Voila le flot des véhicules-----------")
        for i, x in self.flot_voitures.items():
            print(f"""
L'immatriculation : {x.immatriculation}
Marque : {x.marque}
Modèle : {x.modele}
Kilométrage : {x.kilometrage} km
Statut : {x.statut}
Classe : {x.__class__.__name__}
            """)
            print("-------------------------------------------------------")

    def ajouteNouveauClient(self):
        print("veuillez saisir les informations du nouveau client : ")
        nom = input("veuillez saisir le nom du client :  ").strip().capitalize()
        numero_permis = input("veuillez saisir le numero de permis du client :  ").strip().upper()

        if numero_permis in self.clients:
            print("Client deja existant")
            return
        client = Client(nom, numero_permis)
        self.clients[numero_permis] = client
        print("client ajouté avec succès")
        return client

    def afficherClients(self):
        print("-------------Voila la list  des clients -----------")
        for i, x in self.clients.items():
            print(f"""
Le nom  : {x.nom}
Numero de permis  : {x.numero_permis}
            """)
            print("-------------------------------------------------------")

    def afficherVehiculeDisponible(self, classe_filtre=None):
        print("-------------Véhicules Disponibles-------------")
        trouve = False
        for v in self.flot_voitures.values():
            if v.statut == "Disponible":
                # si filtre appliqué on teste la nature de l'objet de quel  class
                if classe_filtre is None or isinstance(v, classe_filtre):
                    print(f"{v.marque} {v.modele} ({v.immatriculation}) - {v.kilometrage} km")
                    trouve = True
        if not trouve:
            print("Aucun véhicule disponible pour ce filtre.")
        print("------------------------------------------------")

    def chercherClient(self, key_word):
      key_word = key_word.strip()
      clients_trouves = []

      for client in self.clients.values():
          if key_word.capitalize() == client.nom or key_word.upper() == client.numero_permis:
              clients_trouves.append(client)

      if clients_trouves:
          print("\n=== Clients trouvés ===")
          for client in clients_trouves:
              print(f"""
  Le nom  : {client.nom}
  Numéro de permis  : {client.numero_permis}
              """)
      else:
          print("Aucun client trouvé.")

      return clients_trouves


    def chercherClientParPermis(self, key_word):
      client_trouve = None
      for i,client in self.clients.items():

        if key_word == client.numero_permis:
            client_trouve = client
        else:
          print("Aucun client trouvé")
      return client_trouve

    def louerVehicule(self):
        immatriculation = input("Veuillez saisir l'immatriculation du véhicule : ").strip().upper()
        key_word = input("Veuillez saisir le numéro de permis : ").strip().upper()

        while True:
          date_debut = input("Veuillez saisir la date de début de location (jj/mm/aaaa) : ").strip()


          if len(date_debut) == 10 and date_debut[2] == "/" and date_debut[5] == "/":
            jj, mm, aaaa = date_debut.split("/")

            if jj.isdigit() and mm.isdigit() and aaaa.isdigit():
                jj, mm, aaaa = int(jj), int(mm), int(aaaa)

                try:
                    # convert into a date object
                    date_debut = date(aaaa, mm, jj)

                    if 1 <= jj <= 31 and 1 <= mm <= 12 and aaaa >= date.today().year and date_debut  >= date.today():
                        print("Date valide :", date_debut)
                        break
                    else:
                        print("Entrez une date valide")
                except ValueError:
                    print("Date invalide")
                    continue
          else:
              print("Format invalide, veuillez entrer au format jj/mm/aaaa")

        # Vérifier si le client existe
        client_trouve = self.chercherClientParPermis(key_word)
        if client_trouve is None:
            print(" Aucun client trouvé, impossible de continuer la location.")
            return

        # Vérifier si le client a déjà une location active

        for loc in self.location_actives.values():
            #if client_trouve.numero_permis == loc.client.numero_permis:
            if client_trouve.numero_permis == loc.Client.numero_permis:
                print(" Ce client a déjà une location active !")
                return

        # Vérifier si le véhicule existe et est disponible
        vehicule_trouve = self.flot_voitures.get(immatriculation)
        if vehicule_trouve is None:
            print(" Aucun véhicule trouvé avec cette immatriculation.")
            return

        if vehicule_trouve.statut != "Disponible":
            print(" Ce véhicule n'est pas disponible.")
            return

        location = LocationActive(vehicule_trouve, client_trouve, date_debut)
        self.location_actives[vehicule_trouve.immatriculation] = location

        vehicule_trouve.statut = "Loué"

        print(f" Véhicule {vehicule_trouve.marque} {vehicule_trouve.modele} loué avec succès à {client_trouve.nom}.")

    def envoyer_en_maintenance(self, immatriculation):
        vehicule_trouve = self.flot_voitures.get(immatriculation)
        if vehicule_trouve is None:
            print(" Aucun véhicule trouvé avec cette immatriculation.")
            return
        if vehicule_trouve.statut == "Disponible":
          vehicule_trouve.statut = "Maintenance"
          print(f"Véhicule {vehicule_trouve.marque} {vehicule_trouve.modele} envoyé en maintenance.")


    def retour_de_maintenance(self, immatriculation):
        vehicule = self.flot_voitures.get(immatriculation)
        if vehicule is None:
            print(" Aucun véhicule trouvé avec cette immatriculation.")
            return
        if vehicule.statut == "Maintenance":
            vehicule.statut = "Disponible"
            print(f"Véhicule {vehicule.marque} {vehicule.modele} est revenu de maintenance et est maintenant disponible.")
        else:
            print(f"Le véhicule {immatriculation} n’est pas en maintenance.")


    def retourner_vehicule(self, immatriculation, km_retour):
      TARIF_JOURNALIER = 50.00
      TARIF_KM = 0.20
      date_retour = datetime.date.today()

      # 1. Vérifier si le véhicule existe
      vehicule = self.flot_voitures.get(immatriculation)
      if vehicule is None:
          return f"Erreur : Véhicule {immatriculation} introuvable."

      # 2. Vérifier s’il y a une location active
      location_active = self.location_actives.get(immatriculation)
      if location_active is None:
          return f"Erreur : Aucun contrat de location actif pour {immatriculation}."

      # 3. Vérifier le statut du véhicule
      if vehicule.statut != "Loué":
          return f"Erreur : Le véhicule {immatriculation} n’est pas en location."

      # 4. Vérifier le kilométrage
      if int(km_retour) < int(vehicule.kilometrage):
          return f"Erreur : kilométrage retour ({km_retour}) < départ ({vehicule.kilometrage})."

      # 5. Calcul du coût
      km_parcouru = int(km_retour) - int(vehicule.kilometrage)
      #Bach ystocker lina la fin
      #vehicule.kilometrage = int(km_retour)

      if isinstance(location_active.date_debut, str):
          date_debut = datetime.datetime.strptime(location_active.date_debut, "%Y-%m-%d").date()
      else:
          date_debut = location_active.date_debut

      duree_jours = max(1, (date_retour - date_debut).days)
      cout_total = duree_jours * TARIF_JOURNALIER + km_parcouru * TARIF_KM

      # 6. Mise à jour du véhicule
      #vehicule.kilometrage = km_retour
      vehicule.kilometrage = int(km_retour)
      vehicule.statut = "Disponible"

      # 7. Création du contrat
      id_contrat = len(self.historique_contrats) + 1
      contrat = Contrat_Location(
          id_contrat=id_contrat,
          immatriculation_du_vehicule=vehicule.immatriculation,
          id_du_client=location_active.Client.id_client,
          date_debut=date_debut,
          date_retour=date_retour,
          kilometrage_parcouru=km_parcouru,
          cout_total=cout_total
      )
#8. Mise à jour de l’historique et suppression de la location active
      self.historique_contrats[id_contrat] = contrat
      del self.location_actives[immatriculation]
# 9. Retour message de confirmation
      return (f"Contrat {id_contrat} clôturé :\n"
              f"- Durée : {duree_jours} jours\n"
              f"- Km parcourus : {km_parcouru}\n"
              f"- Total : {cout_total:.2f} €\n"
              f"- Véhicule {immatriculation} est maintenant disponible.")

    def generer_rapports_agence(self):


      # Charger les données depuis le fichier JSON
      with open("agence.json", "r") as f:
          data = json.load(f)

      historique_dict = data.get("historique_contrats", {})
      flot_voitures = data.get("flot_voitures", {})

      df_contrats = pd.DataFrame(list(historique_dict.values()))
      df_voitures = pd.DataFrame(list(flot_voitures.values()))

      if df_contrats.empty:
          print(" Aucun contrat enregistré.")
          return

      df_contrats["date_debut"] = pd.to_datetime(df_contrats["date_debut"])
      df_contrats["date_retour"] = pd.to_datetime(df_contrats["date_retour"])

      # Calcul de la durée des locations (au moins 1 jour)
      df_contrats["duree_jours"] = (df_contrats["date_retour"] - df_contrats["date_debut"]).dt.days
      df_contrats["duree_jours"] = df_contrats["duree_jours"].clip(lower=1)

      df_contrats['mois_debut'] = df_contrats['date_debut'].dt.month

      # Revenu total par mois
      df_revenu_mois = df_contrats.groupby('mois_debut')['cout_total'].sum()

      # Joindre contrats avec les infos véhicules
      df = df_contrats.merge(
          df_voitures,
          left_on="immatriculation_du_vehicule",
          right_on="immatriculation",
          how="left"
      )

      # Moyenne des jours de location par type de véhicule
      moyenne_par_type = df.groupby("type")["duree_jours"].mean()

      print("====== Rapport Agence ======")
      print("\n Revenu total par mois :")
      print(df_revenu_mois)

      print("\n Nombre moyen de jours de location par type de véhicule :")
      print(moyenne_par_type)

      # Générer le graphique
      plt.plot(df_revenu_mois.index, df_revenu_mois.values, marker='o', color='red')
      plt.xlabel('Mois')
      plt.ylabel('Revenu (€)')
      plt.title('Revenu par mois')
      plt.grid(True)
      plt.savefig('rapport_locations.png')
      plt.show()
      return

**Interface Utilisateur**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
DATA_FILE = "agence.json"

class AgencePersistante(Agence):
    def sauvegarder(self):
        data = {
            "flot_voitures": {imm: v.to_dict() for imm, v in self.flot_voitures.items()},
            "clients": {perm: c.to_dict() for perm, c in self.clients.items()},
            "location_actives": {imm: loc.to_dict() for imm, loc in self.location_actives.items()},
            "historique_contrats": {cid: contrat.to_dict() for cid, contrat in self.historique_contrats.items()}
        }
        with open(DATA_FILE, "w") as f:
            json.dump(data, f, indent=4)

    def charger(self):
        try:
            with open(DATA_FILE, "r") as f:
                data = json.load(f)
            # reconstruction des objets
            self.flot_voitures = {imm: Vehicule.from_dict(v) for imm, v in data["flot_voitures"].items()}
            self.clients = {perm: Client.from_dict(c) for perm, c in data["clients"].items()}
            self.location_actives = {imm: LocationActive.from_dict(loc, self.flot_voitures, self.clients)
                                     for imm, loc in data["location_actives"].items()}
            self.historique_contrats = {int(cid): Contrat_Location.from_dict(ct)
                                        for cid, ct in data["historique_contrats"].items()}
        except FileNotFoundError:
            print(" Premier lancement : aucun fichier trouvé, agence vide.")

# -------------------------------------------------------
# Exemple CLI
def menu():
    agence = AgencePersistante()
    agence.charger()

    while True:
        print("""
        ====== MENU AGENCE ======
        1. Ajouter un véhicule
        2. Ajouter un client
        3. Louer un véhicule
        4. Retourner un véhicule
        5. Envoyer un véhicule en maintenance
        6. Retourner un véhicule de maintenance
        7. Afficher les véhicules disponibles
        8. Afficher tous les véhicules
        9. Afficher les clients
        10. Chercher un client
        11. Générer les rapports
        12. Sauvegarder & Quitter
        """)
        choix = input("Votre choix : ")

        if choix == "1":
            agence.ajouterNouveauVehicule()

        elif choix == "2":
            agence.ajouteNouveauClient()

        elif choix == "3":
            agence.louerVehicule()

        elif choix == "4":
            imm = input("Immatriculation : ")
            while True:
                kilometre = input("Kilométrage retour : ")
                if not kilometre.isdigit():
                    print("Erreur : Kilométrage doit être un nombre entier.")
                    continue
                km = int(kilometre)
                break
            print(agence.retourner_vehicule(imm, km))

        elif choix == "5":
            imm = input("Immatriculation du véhicule à envoyer en maintenance : ").strip().upper()
            agence.envoyer_en_maintenance(imm)

        elif choix == "6":
            imm = input("Immatriculation du véhicule à remettre en service : ").strip().upper()
            agence.retour_de_maintenance(imm)

        elif choix == "7":
            agence.afficherVehiculeDisponible()

        elif choix == "8":
            agence.afficherFlotVehicules()

        elif choix == "9":
            agence.afficherClients()

        elif choix == "10":
            key_word = input("Entrez le nom ou le numéro de permis du client : ")
            agence.chercherClient(key_word)

        elif choix == "11":
            agence.generer_rapports_agence()

        elif choix == "12":
            agence.sauvegarder()
            print("Sauvegarde avec succès. Au revoir !")
            break

        else:
            print(" Choix invalide")


if __name__ == "__main__":
    menu()


 Premier lancement : aucun fichier trouvé, agence vide.

        ====== MENU AGENCE ======
        1. Ajouter un véhicule
        2. Ajouter un client
        3. Louer un véhicule
        4. Retourner un véhicule
        5. Envoyer un véhicule en maintenance
        6. Retourner un véhicule de maintenance
        7. Afficher les véhicules disponibles
        8. Afficher tous les véhicules
        9. Afficher les clients
        10. Chercher un client
        11. Générer les rapports
        12. Sauvegarder & Quitter
        
veuillez saisir les informations du nouveau vehicule : 
 Véhicule ajouté avec succès

        ====== MENU AGENCE ======
        1. Ajouter un véhicule
        2. Ajouter un client
        3. Louer un véhicule
        4. Retourner un véhicule
        5. Envoyer un véhicule en maintenance
        6. Retourner un véhicule de maintenance
        7. Afficher les véhicules disponibles
        8. Afficher tous les véhicules
        9. Afficher les clients
        10. Cher